### Install Python Libraries

In [1]:
# Check if you're working in venv
import sys
sys.executable

'C:\\Users\\Kalis\\Documents\\RAG\\Project - Implementing RAG with Llama\\myenv\\Scripts\\python.exe'

- We will be using langchain to build our RAG and connect it to llama3.1

In [2]:
!pip install langchain langchain_community langchain-openai scikit-learn langchain-ollama

  Using cached langchain-1.0.8-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langgraph-1.0.3-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain-1.0.8-py3-none-any.whl (93 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached langgraph-1.0.3-py3-none-any.whl (156 kB)



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Load Documents

- WebBaseLoader is used to fetch content from each URL provided. The resulting nested lists of documents are then combined into a single, flat list called docs_list, giving us a list of documents.

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# List of URLs to load documents from
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

### Split Document in Chunks

In [5]:
# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)

### Convert Text into embeddings and store in vector store

In [7]:
!pip install sentence-transformers

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.0 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.0 kB 330.3 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 44.0/44.0 kB 309.6 kB/s eta 0:00:00
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/488.0 kB ? eta -:--:--
   --- ----------------------------------- 41.0/488.0 kB 991.0 kB/s eta 0:00:01
   ---- ---------------------------------- 61.4/488.0 kB 825.8 kB/s eta 0:00:01
   ----- --------------------------------- 71.7/488.0 kB 660.6 kB/s eta 0:00:01
   -------------- ------------------------- 174.1/488.0 kB 1.1 MB/s eta 0:00:01
   ---------------- ----------------------- 204.8/488.0 kB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 256.0/488.0 kB 1.1 MB/s eta 0:00:0


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
# Create embeddings for documents and store them in a vector store
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=HuggingFaceEmbeddings()
)
retriever = vectorstore.as_retriever(k=4)

C:\Users\Kalis\AppData\Local\Temp\ipykernel_18312\42725836.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings()
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-mpnet-base-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000269F411FA40>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: f9e5dbd4-5005-4813-bfcc-985153e54cc3)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-mpnet-base-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transfor

### Set up the LLM and prompt template

In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

#### Initialize model

In [ ]:
# Initialize the LLM with Llama 3.1 model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)

In [ ]:
# Create a chain combining the prompt template and LLM
rag_chain = prompt | llm | StrOutputParser()

### Integrate the retriever and LLM into a RAG application

In [ ]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Retrieve relevant documents
        documents = self.retriever.invoke(question)
        # Extract content from retrieved documents
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        # Get the answer from the language model
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

### Test Application

In [ ]:
# Initialize the RAG application
rag_application = RAGApplication(retriever, rag_chain)
# Example usage
question = "What is prompt engineering"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)